In [21]:
from api import call_deepseek_api

In [22]:
def type_choose(input):
    """
    Функция определяет тип запроса
    """
    router_template = """\
    Вам нужно классифицировать пользовательский запрос по одному из трёх типов:
    1) TYPE_1 — “Создать диаграмму с нуля по описанию”
    2) TYPE_2 — “Добавить элемент в уже существующую диаграмму”
    3) TYPE_3 — “Редактировать диаграмму по набору рекомендаций или новым правкам от пользователя”

    Отвечайте строго одним кодовым словом: TYPE_1, TYPE_2 или TYPE_3.

    Пример:
    Вопрос: “…”  
    Ответ: TYPE_2

    Вопрос: {input}  
    Ответ:"""

    final_promt = router_template.format(input=input)
    return final_promt

In [23]:
"""
Промты и дефолтные бизнесс требования
"""

default_business_requirements = "1. Схема должна быть грамотная и удобная для чтения. 2. Если возможно какой-то комплексный блок разбить на меньшие блоки - сделай это"

templates = {
    'TYPE_1': """
                **Роль:** Ты — продвинутый ИИ-ассистент, специализирующийся на анализе и моделировании бизнес-процессов в формате BPMN.

                **Задача:** Проанализируй следующее текстовое описание бизнес-процесса и преобразуй его в структурированный текст, используя синтаксис PiperFlow, который подходит для библиотеки Python `processpiper`. Твоя цель — максимально точно воспроизвести структуру и элементы, показанные в примере ниже.

                **Входное описание процесса:**
                {user_prompt}

                **Бизнес требования к диаграмме**
                {buisness}

                **Требования к выходному формату (PiperFlow Syntax):**

                Вы — помощник по преобразованию описания бизнес‑процесса в синтаксис PiperFlow для библиотеки processpiper. 
                Когда приходит текстовое описание, выдавайте только блок кода, строго по этому шаблону, не используя двойные кавчки в ответе:

                title: <Название процесса>
                colourtheme: BLUEMOUNTAIN

                pool: <Имя первого пула>
                    lane: <Имя первой роли>
                        (start) as start_event
                        [<Действие 1>] as <идентификатор_1>
                        [<Действие 2>] as <идентификатор_2>
                        …
                        start_event -> <идентификатор_1> -> <идентификатор_2> -> …

                pool: <Имя второго пула>
                    lane: <Имя роли>
                        [<Действие 3>] as <идентификатор_3>
                        <Условие?> as <идентификатор_шлюза>  
                        [<Действие 4>] as <идентификатор_4>
                        [<Действие 5>] as <идентификатор_5>
                        (end) as end_event

                        <поток_событий_1>
                        <поток_событий_2>
                        …

                footer: <Краткое описание процесса>

                **Полный пример желаемого вывода (используй этот формат и стиль):**
                """,
    "TYPE_2": """
            **Роль:** Ты — продвинутый ИИ-ассистент, специализирующийся на анализе и моделировании бизнес-процессов в формате BPMN.

            **Задача:** Проанализируй следующее текстовое описание и добавь новые элементы диаграммы, используя синтаксис PiperFlow, который подходит для библиотеки Python `processpiper`. Твоя цель — не исправлять имеющуюся диаграмму, а добавить нужный элемент.

            **Входное описание процесса:**
            Запрос пользователя: {user_prompt}
            Ранее задействованный PiperFlow синтаксис: {piperflow}

            **Бизнес требования к диаграмме**
                {buisness}

            **Требования к выходному формату (PiperFlow Syntax):**

            Вы — помощник по преобразованию описания бизнес‑процесса в синтаксис PiperFlow для библиотеки processpiper. 
            Когда приходит текстовое описание, выдавайте только блок кода, строго по этому шаблону, не используя двойные кавчки в ответе:

            title: <Название процесса>
            colourtheme: BLUEMOUNTAIN

            pool: <Имя первого пула>
                lane: <Имя первой роли>
                    (start) as start_event
                    [<Действие 1>] as <идентификатор_1>
                    [<Действие 2>] as <идентификатор_2>
                    …
                    start_event -> <идентификатор_1> -> <идентификатор_2> -> …

            pool: <Имя второго пула>
                lane: <Имя роли>
                    [<Действие 3>] as <идентификатор_3>
                    <Условие?> as <идентификатор_шлюза>  
                    [<Действие 4>] as <идентификатор_4>
                    [<Действие 5>] as <идентификатор_5>
                    (end) as end_event

                    <поток_событий_1>
                    <поток_событий_2>
                    …

            footer: <Краткое описание процесса>

            **Полный пример желаемого вывода (используй этот формат и стиль):**
            """,

    'TYPE_3':"""
            **Роль:** Ты — продвинутый ИИ-ассистент, специализирующийся на анализе и моделировании бизнес-процессов в формате BPMN.

            **Задача:** Проанализируй следующее текстовое описание и отредактируй ее по правкам, используя синтаксис PiperFlow, который подходит для библиотеки Python `processpiper`. Твоя цель — создать лучшую диаграмму на основе правок.

            **Входное описание процесса:**
            Запрос пользователя: {user_prompt}
            Ранее задействованный PiperFlow синтаксис: {piperflow}
            Рекомендации : {recs}

            **Бизнес требования к диаграмме**
                {buisness}

            **Требования к выходному формату (PiperFlow Syntax):**

            Вы — помощник по преобразованию описания бизнес‑процесса в синтаксис PiperFlow для библиотеки processpiper. 
            Когда приходит текстовое описание, выдавайте только блок кода, строго по этому шаблону, не используя двойные кавчки в ответе:

            title: <Название процесса>
            colourtheme: BLUEMOUNTAIN

            pool: <Имя первого пула>
                lane: <Имя первой роли>
                    (start) as start_event
                    [<Действие 1>] as <идентификатор_1>
                    [<Действие 2>] as <идентификатор_2>
                    …
                    start_event -> <идентификатор_1> -> <идентификатор_2> -> …

            pool: <Имя второго пула>
                lane: <Имя роли>
                    [<Действие 3>] as <идентификатор_3>
                    <Условие?> as <идентификатор_шлюза>  
                    [<Действие 4>] as <идентификатор_4>
                    [<Действие 5>] as <идентификатор_5>
                    (end) as end_event

                    <поток_событий_1>
                    <поток_событий_2>
                    …

            footer: <Краткое описание процесса>

            **Полный пример желаемого вывода (используй этот формат и стиль):**
            """
}

In [24]:
def promt_teplate_creator_and_answer(user_promt: str, buisness:str, recomendations:str = None, piperflow_text: str = None):
    """
    Функция отдает промт по типу запроса
    """
    if piperflow_text == None:
        promt = templates['TYPE_1'].format(user_prompt=user_promt, buisness=buisness)
    else:
        promt_type = type_choose(user_promt)
        if promt_type == 'TYPE_1':
            promt = templates['TYPE_1'].format(user_prompt=user_promt, buisness=buisness)
        elif promt_type == 'TYPE_2':
            promt = templates['TYPE_2'].format(user_prompt=user_promt, piperflow=piperflow_text, buisness=buisness)
        else:
            promt = templates['TYPE_3'].format(user_prompt=user_promt, piperflow=piperflow_text, recs=recomendations, buisness=buisness)
    answer = call_deepseek_api(promt)
    return answer

In [25]:
"""
Пример как создать с нуля PiperFlow
"""
user_promt = "Процесс начинается, когда клиент нажимает кнопку 'Оформить заказ' на сайте. Система должна проверить наличие товара на складе. Если товар в наличии, нужно создать счет на оплату и отправить его клиенту. Если товара нет, необходимо уведомить клиента об отсутствии и предложить похожие товары. После отправки счета или уведомления процесс завершается."

answer = promt_teplate_creator_and_answer(user_promt=user_promt, buisness=default_business_requirements)
print(answer)


=== API Request ===
Model: deepseek-ai/DeepSeek-V3-0324
Temperature: 0.1
Max tokens: 4096

Prompt:

        You are a BPMN expert. Generate a complete and valid BPMN 2.0 XML diagram for the following business process description.
        
        IMPORTANT: The XML MUST start exactly with this header (copy it exactly):
        <?xml version="1.0" encoding="UTF-8"?>
        <bpmn2:definitions 
            xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
            xmlns:bpmn2="http://www.omg.org/spec/BPMN/20100524/MODEL"
            xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI"
            xmlns:dc="http://www.omg.org/spec/DD/20100524/DC"
            xmlns:di="http://www.omg.org/spec/DD/20100524/DI"
            id="Definitions_fe5e701f"
            targetNamespace="http://bpmn.io/schema/bpmn">

        Rules for XML generation:
        1. Use ONLY bpmn2: prefix for BPMN elements (not bpmn:)
        2. All process elements must be inside <bpmn2:process>
        3. All di

In [26]:
"""
Пример как получить нормальный PiperFlow перед подачей в библиотеку
"""
import re

def extract_piperflow_block(text: str) -> str:
    """
    Ищет в тексте блок, начинающийся с 'title:' и заканчивающийся на 'colourtheme:',
    включая тему (например, 'colourtheme: BLUEMOUNTAIN').
    Возвращает найденный блок или пустую строку, если не найдено.
    """
    # Регэксп: захватывает минимальный блок от title: до последнего colourtheme:
    text = text.replace('"', '')
    pattern = """{text}
    """
    final_text = pattern.format(text=text)
    return final_text

piperflow = extract_piperflow_block(answer)

In [27]:
def recs_generation(piperflow:str, current_process:str, buisness:str):
    """
    Генерирует рекомендации по улучшению диаграмм с точки зрения бизнеса
    """
    promt = """
    **Роль:** 
    Ты — продвинутый ИИ-ассистент, специализирующийся на анализе и моделировании бизнес-процессов в формате BPMN.

    **Задача:** Проанализируй следующее текстовое описание и отредактируй ее по правкам, используя синтаксис PiperFlow, который подходит для библиотеки Python `processpiper`. Твоя цель — написать рекомендации по улучшению диаграммы с точки зрения бизнеса, не пиши код Piperflow

    **Входное описание процесса:**
    Текущий PiperFlow синтаксис: {piperflow}

    **Описание бизнес-процесса**
    Текущий бизнесс процесс: {current_process}

    **Бизнес требования**
    Требования со стороны бизнеса: {buisness}

    Улучши эту схему, основываясь на бизнес тробованиях.
    """
    promt = promt.format(piperflow=piperflow, current_process=current_process, buisness=buisness)
    recs = call_deepseek_api(promt)

    return recs

In [28]:
"""
Пример как получить рекомендации по имеющейся диаграмме
"""
recs = recs_generation(piperflow, user_promt, default_business_requirements)
print(recs)


=== API Request ===
Model: deepseek-ai/DeepSeek-V3-0324
Temperature: 0.1
Max tokens: 4096

Prompt:

        You are a BPMN expert. Generate a complete and valid BPMN 2.0 XML diagram for the following business process description.
        
        IMPORTANT: The XML MUST start exactly with this header (copy it exactly):
        <?xml version="1.0" encoding="UTF-8"?>
        <bpmn2:definitions 
            xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
            xmlns:bpmn2="http://www.omg.org/spec/BPMN/20100524/MODEL"
            xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI"
            xmlns:dc="http://www.omg.org/spec/DD/20100524/DC"
            xmlns:di="http://www.omg.org/spec/DD/20100524/DI"
            id="Definitions_dd8a481c"
            targetNamespace="http://bpmn.io/schema/bpmn">

        Rules for XML generation:
        1. Use ONLY bpmn2: prefix for BPMN elements (not bpmn:)
        2. All process elements must be inside <bpmn2:process>
        3. All di

In [29]:
"""
Пример добавления блока
"""
user_promt = "Добавь в данную диаграмму блок Оплата заказа в линии клиента после нажатия кнопки оформить заказ"

answer = promt_teplate_creator_and_answer(user_promt=user_promt, piperflow_text=piperflow, buisness=default_business_requirements)
print(answer)


=== API Request ===
Model: deepseek-ai/DeepSeek-V3-0324
Temperature: 0.1
Max tokens: 4096

Prompt:

        You are a BPMN expert. Generate a complete and valid BPMN 2.0 XML diagram for the following business process description.
        
        IMPORTANT: The XML MUST start exactly with this header (copy it exactly):
        <?xml version="1.0" encoding="UTF-8"?>
        <bpmn2:definitions 
            xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
            xmlns:bpmn2="http://www.omg.org/spec/BPMN/20100524/MODEL"
            xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI"
            xmlns:dc="http://www.omg.org/spec/DD/20100524/DC"
            xmlns:di="http://www.omg.org/spec/DD/20100524/DI"
            id="Definitions_cebb2cfe"
            targetNamespace="http://bpmn.io/schema/bpmn">

        Rules for XML generation:
        1. Use ONLY bpmn2: prefix for BPMN elements (not bpmn:)
        2. All process elements must be inside <bpmn2:process>
        3. All di

In [30]:
"""
Пример улучшения с рекомендациями
"""
answer = promt_teplate_creator_and_answer(user_promt=user_promt, piperflow_text=piperflow, recomendations=recs, buisness=default_business_requirements)
print(answer)


=== API Request ===
Model: deepseek-ai/DeepSeek-V3-0324
Temperature: 0.1
Max tokens: 4096

Prompt:

        You are a BPMN expert. Generate a complete and valid BPMN 2.0 XML diagram for the following business process description.
        
        IMPORTANT: The XML MUST start exactly with this header (copy it exactly):
        <?xml version="1.0" encoding="UTF-8"?>
        <bpmn2:definitions 
            xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
            xmlns:bpmn2="http://www.omg.org/spec/BPMN/20100524/MODEL"
            xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI"
            xmlns:dc="http://www.omg.org/spec/DD/20100524/DC"
            xmlns:di="http://www.omg.org/spec/DD/20100524/DI"
            id="Definitions_3ccd2d87"
            targetNamespace="http://bpmn.io/schema/bpmn">

        Rules for XML generation:
        1. Use ONLY bpmn2: prefix for BPMN elements (not bpmn:)
        2. All process elements must be inside <bpmn2:process>
        3. All di

In [31]:
from processpiper.text2diagram import render


def diagramm_creator(piperflow_text):
    def diagramm(piperflow_text):
        output_file_name_ru_v3 = "process_example_corrected_ru_v3.png"
        try:
            render(piperflow_text, output_file_name_ru_v3)
            print(f"Исправленная диаграмма V3 (RU) сохранена в файл {output_file_name_ru_v3}.png")
            return '200: OK'
        except ValueError as e:
            return e
        except Exception as e: 
            return e
        
    diagramm_answer = diagramm(piperflow_text)
    if diagramm_answer == '200: OK':
        return piperflow_text
    
    else:
        promt = """
        Вот твой PiperFlow:
        {piperflow}

        При генерации диаграммы BPNM библиотекой processpiper вышла ошибка: {exception}
        Переделай PiperFlow, чтобы ошибки не было, при этом не удаляй уже имеющиеся блоки
        Твой ответ должен содержать только текст PiperFlow вида

        title: <Название процесса>
            colourtheme: BLUEMOUNTAIN

            pool: <Имя первого пула>
                lane: <Имя первой роли>
                    (start) as start_event
                    [<Действие 1>] as <идентификатор_1>
                    [<Действие 2>] as <идентификатор_2>
                    …
                    start_event -> <идентификатор_1> -> <идентификатор_2> -> …

            pool: <Имя второго пула>
                lane: <Имя роли>
                    [<Действие 3>] as <идентификатор_3>
                    <Условие?> as <идентификатор_шлюза>  
                    [<Действие 4>] as <идентификатор_4>
                    [<Действие 5>] as <идентификатор_5>
                    (end) as end_event

                    <поток_событий_1>
                    <поток_событий_2>
                    …

            footer: <Краткое описание процесса>
        """
        promt = promt.format(piperflow=piperflow_text, exception=diagramm_answer)
        answer = call_deepseek_api(final_promt=promt)
        piperflow_text = diagramm_creator(answer)

In [32]:
diagramm_creator(piperflow)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 diagramm_creator(piperflow)                                                                  │
│   2                                                                                              │
│                                                                                                  │
│ in diagramm_creator:55                                                                           │
│                                                                                                  │
│   52 │   │   │   footer: <Краткое описание процесса>                                             │
│   53 │   │   """                                                                                 │
│   54 │   │   promt = promt.format(piperflow=piperflow_text, exception=diagramm_answer)           │
│ ❱ 55 │   │   answer = call_deepseek_api(final_promt=promt)                                       │
│   56 │   │   piperflow_text = diagramm_creator(answer)                                           │
│   57                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: call_deepseek_api() got an unexpected keyword argument 'final_promt'